# Initialisation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OrdinalEncoder #, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

from FeatureNames import get_feature_names

# Exploration

In [2]:
! ls -lh ../02_data/

total 2,5G
-rw-rw-r-- 1 adrien adrien  26M juin  26  2018 application_test.csv
-rw-rw-r-- 1 adrien adrien 159M juin  26  2018 application_train.csv
-rw-rw-r-- 1 adrien adrien 359M juin  26  2018 bureau_balance.csv
-rw-rw-r-- 1 adrien adrien 163M juin  26  2018 bureau.csv
-rw-rw-r-- 1 adrien adrien 405M juin  26  2018 credit_card_balance.csv
-rw-rw-r-- 1 adrien adrien  37K juin  26  2018 HomeCredit_columns_description.csv
-rw-rw-r-- 1 adrien adrien 690M juin  26  2018 installments_payments.csv
-rw-rw-r-- 1 adrien adrien 375M juin  26  2018 POS_CASH_balance.csv
-rw-rw-r-- 1 adrien adrien 387M juin  26  2018 previous_application.csv
-rw-rw-r-- 1 adrien adrien 524K juin  26  2018 sample_submission.csv


In [3]:
col_desc = pd.read_csv('../02_data/HomeCredit_columns_description.csv', 
                       index_col=0)
col_desc

,Table,Row,Description,Special
1,application_{train|test}.csv,SK_ID_CURR,ID of loan in our sample,NaN
2,application_{train|test}.csv,TARGET,Target variable (1 - client with payment diffi...,NaN
5,application_{train|test}.csv,NAME_CONTRACT_TYPE,Identification if loan is cash or revolving,NaN
6,application_{train|test}.csv,CODE_GENDER,Gender of the client,NaN
7,application_{train|test}.csv,FLAG_OWN_CAR,Flag if the client owns a car,NaN
...,...,...,...,...
217,installments_payments.csv,NUM_INSTALMENT_NUMBER,On which installment we observe payment,NaN
218,installments_payments.csv,DAYS_INSTALMENT,When the installment of previous credit was su...,time only relative to the application
219,installments_payments.csv,DAYS_ENTRY_PAYMENT,When was the installments of previous credit p...,time only relative to the application
220,installments_payments.csv,AMT_INSTALMENT,What was the prescribed installment amount of ...,NaN


## Tables `application_{train|test}.csv`

Il y a plus de 200 colonnes pour 9 tables au format csv !
Avant d'aller plus loin dans l'exploration je vais me concentrer sur les tables principales : les tables `application_{train|test}.csv`.

Je vais d'abord regarder les plus grosses corrélations avec la variable `TARGET`

In [4]:
app_train = pd.read_csv('../02_data/application_train.csv')
app_test = pd.read_csv('../02_data/application_test.csv')
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
assert len(app_train.SK_ID_CURR.unique()) == app_train.shape[0]
assert len(app_test.SK_ID_CURR.unique()) == app_test.shape[0]

app_train.set_index('SK_ID_CURR', inplace=True)
app_test.set_index('SK_ID_CURR', inplace=True)

app_train.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
print('name_col' + '\t' + 'data_type' + '\t' + 'dimensionality' + '\t' 
      + 'null_count' + '\t' + 'null_perct' + '\t'+ 'description')
for col in app_train.columns.tolist():
    column_typ = app_train[col].dtypes
    null_count = app_train[col].isna().sum()
    null_perct = null_count / app_train[col].isna().count()
    if app_train[col].dtype in ['object', 'int64']:
        dimensionality = app_train[col].nunique()
    else:
        dimensionality = np.nan
    desc = col_desc.loc[col_desc.Table.eq('application_{train|test}.csv')
                        & col_desc.Row.eq(col)].Description.tolist()[0]
    print(col + '\t'
          + str(column_typ) + '\t'
          + str(dimensionality) + '\t'
          + str(null_count) + '\t'
          + str(round(null_perct, 4) * 100) + '\t'
          + str(desc))

name_col	data_type	dimensionality	null_count	null_perct	description
TARGET	int64	2	0	0.0	Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)
NAME_CONTRACT_TYPE	object	2	0	0.0	Identification if loan is cash or revolving
CODE_GENDER	object	3	0	0.0	Gender of the client
FLAG_OWN_CAR	object	2	0	0.0	Flag if the client owns a car
FLAG_OWN_REALTY	object	2	0	0.0	Flag if client owns a house or flat
CNT_CHILDREN	int64	15	0	0.0	Number of children the client has
AMT_INCOME_TOTAL	float64	nan	0	0.0	Income of the client
AMT_CREDIT	float64	nan	0	0.0	Credit amount of the loan
AMT_ANNUITY	float64	nan	12	0.0	Loan annuity
AMT_GOODS_PRICE	float64	nan	278	0.09	For consumer loans it is the price of the goods for which the loan is given
NAME_TYPE_SUITE	object	7	1292	0.42	Who was accompanying client when he was applying for the loan
NAME_INCOME_TYPE	object	8	0	0.0	Clients incom

### Encodage des colonnes textuelles

In [7]:
print(app_train.dtypes.value_counts())

float64    65
int64      40
object     16
dtype: int64


In [8]:
print(app_train.select_dtypes('object').apply(pd.Series.nunique, axis=0))

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64


In [9]:
categorical_labels = app_train.select_dtypes('object').columns.tolist()
for col in categorical_labels:
    null_count = app_train[col].isna().sum()
    null_perct = null_count / app_train[col].isna().count()
    if null_count != 0:
        print(col, null_count, round(null_perct, 4))

NAME_TYPE_SUITE 1292 0.0042
OCCUPATION_TYPE 96391 0.3135
FONDKAPREMONT_MODE 210295 0.6839
HOUSETYPE_MODE 154297 0.5018
WALLSMATERIAL_MODE 156341 0.5084
EMERGENCYSTATE_MODE 145755 0.474


In [11]:
print(app_train.OCCUPATION_TYPE.unique())

['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Laborers' 'Realty agents' 'Secretaries'
 'IT staff' 'HR staff']


In [12]:
desc = col_desc.loc[col_desc.Table.eq('application_{train|test}.csv')
                    & col_desc.Row.eq(col)].Description.tolist()

['What kind of occupation does the client have']


In [53]:
app_train.OCCUPATION_TYPE.fillna('Unknown', inplace=True)

In [20]:
app_train.dropna(subset=['NAME_TYPE_SUITE'], inplace=True)

In [21]:
app_train.NAME_TYPE_SUITE.unique()

array(['Unaccompanied', 'Family', 'Spouse, partner', 'Children',
       'Other_A', 'Other_B', 'Group of people'], dtype=object)

In [29]:
print(col_desc.loc[col_desc.Table.eq('application_{train|test}.csv')
                   & col_desc.Row.eq('CODE_GENDER')].Description.tolist())

['Gender of the client']


In [13]:
app_train.FONDKAPREMONT_MODE

SK_ID_CURR
100002    reg oper account
100003    reg oper account
100004                 NaN
100006                 NaN
100007                 NaN
                ...       
456251    reg oper account
456252    reg oper account
456253    reg oper account
456254                 NaN
456255                 NaN
Name: FONDKAPREMONT_MODE, Length: 307511, dtype: object

In [13]:
app_train.drop(columns=['FONDKAPREMONT_MODE',
                        'HOUSETYPE_MODE',
                        'WALLSMATERIAL_MODE',
                        'EMERGENCYSTATE_MODE'], inplace=True)

In [32]:
categ_const_imput_prep = make_pipeline(SimpleImputer(strategy='constant',
                                                     fill_value='Unknown'),
                                       OrdinalEncoder())

categ_const_imput_cols = ['OCCUPATION_TYPE', 'WALLSMATERIAL_MODE',
                          'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE',
                          'EMERGENCYSTATE_MODE']

categ_mfreq_imput_prep = make_pipeline(SimpleImputer(strategy='most_frequent'),
                                       OrdinalEncoder())

categ_mfreq_imput_cols = ['NAME_TYPE_SUITE']

weekdays = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY',
            'FRIDAY', 'SATURDAY', 'SUNDAY']
# * cos(2 * np.pi) / 7 # pour garder la notion de cyclicité
# * sin(2 * np.pi) / 7
educ_typ = ['Lower secondary', 'Secondary / secondary special', 
            'Incomplete higher', 'Higher education', 'Academic degree']
ordinal_maps = [weekdays, educ_typ]

categ_ordinal_map_prep = make_pipeline(OrdinalEncoder(categories=ordinal_maps))

categ_ordinal_map_cols = ['WEEKDAY_APPR_PROCESS_START', 'NAME_EDUCATION_TYPE']

categ_separate_prep_cols = categ_const_imput_cols + categ_mfreq_imput_cols\
                           + categ_ordinal_map_cols

categorical_labels = app_train.select_dtypes('object').columns.tolist()
preprocessor = make_column_transformer(
    (categ_const_imput_prep, categ_const_imput_cols),
    (categ_mfreq_imput_prep, categ_mfreq_imput_cols),
    (categ_ordinal_map_prep, categ_ordinal_map_cols),
    (OrdinalEncoder(), [c for c in categorical_labels
                        if c not in categ_separate_prep_cols]),
    remainder='passthrough'
)

In [33]:
app_train_processed = preprocessor.fit_transform(app_train)

In [57]:
app_train.REO_.value_counts()

AttributeError: 'DataFrame' object has no attribute 'AMT_REO_CREDIT_BUREAU_HOUR'

In [34]:
feature_names = get_feature_names(preprocessor)

In [35]:
print(feature_names)

['pipeline-1__OCCUPATION_TYPE', 'pipeline-1__WALLSMATERIAL_MODE', 'pipeline-1__FONDKAPREMONT_MODE', 'pipeline-1__HOUSETYPE_MODE', 'pipeline-1__EMERGENCYSTATE_MODE', 'pipeline-2__NAME_TYPE_SUITE', 'pipeline-3__WEEKDAY_APPR_PROCESS_START', 'pipeline-3__NAME_EDUCATION_TYPE', 'ordinalencoder__NAME_CONTRACT_TYPE', 'ordinalencoder__CODE_GENDER', 'ordinalencoder__FLAG_OWN_CAR', 'ordinalencoder__FLAG_OWN_REALTY', 'ordinalencoder__NAME_INCOME_TYPE', 'ordinalencoder__NAME_FAMILY_STATUS', 'ordinalencoder__NAME_HOUSING_TYPE', 'ordinalencoder__ORGANIZATION_TYPE', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'RE

In [36]:
new_feature_names = [name.split('__')[-1] for name in feature_names]
print(new_feature_names)

['OCCUPATION_TYPE', 'WALLSMATERIAL_MODE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE', 'NAME_TYPE_SUITE', 'WEEKDAY_APPR_PROCESS_START', 'NAME_EDUCATION_TYPE', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'ORGANIZATION_TYPE', 'TARGET', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_AVG', '

In [37]:
df = pd.DataFrame(app_train_processed, columns = new_feature_names)

In [38]:
df

,OCCUPATION_TYPE,WALLSMATERIAL_MODE,FONDKAPREMONT_MODE,HOUSETYPE_MODE,EMERGENCYSTATE_MODE,NAME_TYPE_SUITE,WEEKDAY_APPR_PROCESS_START,NAME_EDUCATION_TYPE,NAME_CONTRACT_TYPE,CODE_GENDER,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,8.0,5.0,3.0,1.0,0.0,6.0,2.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3.0,0.0,3.0,1.0,0.0,1.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8.0,6.0,0.0,0.0,1.0,6.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8.0,6.0,0.0,0.0,1.0,6.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,3.0,6.0,0.0,0.0,1.0,6.0,3.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,14.0,5.0,3.0,1.0,0.0,6.0,3.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
307507,17.0,5.0,3.0,1.0,0.0,6.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
307508,10.0,4.0,3.0,1.0,0.0,6.0,3.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
307509,8.0,5.0,0.0,1.0,0.0,6.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Étude des variables

In [39]:
feat_corr = df.corr()['TARGET'].sort_values()

In [40]:
print(feat_corr[:10])
print(feat_corr[-10:])

EXT_SOURCE_3                 -0.178919
EXT_SOURCE_2                 -0.160472
EXT_SOURCE_1                 -0.155317
NAME_EDUCATION_TYPE          -0.056872
DAYS_EMPLOYED                -0.044932
FLOORSMAX_AVG                -0.044003
FLOORSMAX_MEDI               -0.043768
FLOORSMAX_MODE               -0.043226
AMT_GOODS_PRICE              -0.039645
REGION_POPULATION_RELATIVE   -0.037227
Name: TARGET, dtype: float64
FLAG_EMP_PHONE                 0.045982
NAME_INCOME_TYPE               0.046829
REG_CITY_NOT_WORK_CITY         0.050994
DAYS_ID_PUBLISH                0.051457
CODE_GENDER                    0.054692
DAYS_LAST_PHONE_CHANGE         0.055218
REGION_RATING_CLIENT           0.058899
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_BIRTH                     0.078239
TARGET                         1.000000
Name: TARGET, dtype: float64


In [26]:
app_train.EXT_SOURCE_1.dtype

dtype('float64')

In [40]:
app_train.NAME_FAMILY_STATUS.value_counts()

Married                 196432
Single / not married     45444
Civil marriage           29775
Separated                19770
Widow                    16088
Unknown                      2
Name: NAME_FAMILY_STATUS, dtype: int64

In [41]:
app_train.NAME_EDUCATION_TYPE.value_counts()

Secondary / secondary special    218391
Higher education                  74863
Incomplete higher                 10277
Lower secondary                    3816
Academic degree                     164
Name: NAME_EDUCATION_TYPE, dtype: int64

In [54]:
app_train.NAME_TYPE_SUITE.value_counts()

Unaccompanied      248526
Family              40149
Spouse, partner     11370
Children             3267
Other_B              1770
Other_A               866
Group of people       271
Name: NAME_TYPE_SUITE, dtype: int64

In [65]:
app_train['AGE'] = round(app_train['DAYS_BIRTH'] / - 365, 0).astype('int')

In [66]:
app_train.AGE

0         26
1         46
2         52
3         52
4         55
          ..
307506    26
307507    57
307508    41
307509    33
307510    46
Name: AGE, Length: 307511, dtype: int64

## Autres tables

In [22]:
col_desc.loc[col_desc.Table == 'bureau_balance.csv'].Description.values

array(['Recoded ID of Credit Bureau credit (unique coding for each application) - use this to join to CREDIT_BUREAU table ',
       'Month of balance relative to application date (-1 means the freshest balance date)',
       'Status of Credit Bureau loan during the month (active, closed, DPD0-30,� [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,� 5 means DPD 120+ or sold or written off ] )'],
      dtype=object)

In [12]:
bureau = pd.read_csv('../02_data/bureau.csv')
bureau_balance = pd.read_csv('../02_data/bureau_balance.csv')

In [13]:
bureau_balance.shape

(27299925, 3)

In [15]:
bureau_balance.shape[0] / 10 ** 3

27299.925

In [14]:
bureau_balance.columns

Index(['SK_ID_BUREAU', 'MONTHS_BALANCE', 'STATUS'], dtype='object')

In [24]:
bureau.shape

(1716428, 17)

In [20]:
bureau = pd.read_csv('../02_data/bureau.csv')
bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN


In [22]:
bureau.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')

In [32]:
col_desc.loc[col_desc.Row.eq('SK_ID_CURR') & col_desc.Table.eq('bureau.csv')].Description.values

array(['ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau '],
      dtype=object)

In [34]:
bureau.shape

(1716428, 17)

In [35]:
len(bureau.SK_ID_BUREAU.unique())

1716428

In [37]:
len(bureau.SK_ID_CURR.unique())

305811

In [40]:
bureau[bureau.duplicated(subset=['SK_ID_CURR']) == True]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.00,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.00,71017.38,108982.62,0.0,Credit card,-31,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1716423,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.00,0.00,0.0,Microloan,-19,NaN
1716424,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.00,0.00,0.0,Consumer credit,-2493,NaN
1716425,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN
1716426,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.00,0.00,0.0,Consumer credit,-1508,NaN
